## YOLOv8 Tooth Number Detection Training

This notebook trains a YOLOv8 model to detect and classify tooth numbers using the dataset prepared in this project. It:
- installs required dependencies,
- checks your compute device (Apple MPS/CUDA/CPU),
- loads YOLOv8,
- trains on the dataset defined by `data.yaml`, and
- displays training results and basic metrics.


In [3]:
# Install dependencies (ultralytics, opencv-python, matplotlib)
# Note: This cell installs packages into the current environment.
# If you're using a managed kernel, you might not need to run this.
!pip install ultralytics opencv-python matplotlib


In [4]:
# Ensure PyTorch is installed in THIS Jupyter kernel.
# This prints the interpreter path used by the notebook and installs torch/torchvision accordingly.
import sys, subprocess
print("Notebook Python:", sys.version)
print("Interpreter:", sys.executable)

# Install specific versions known to work with Ultralytics used earlier
%pip install --upgrade pip
%pip install torch==2.8.0 torchvision==0.23.0


Notebook Python: 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]
Interpreter: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/b7/3f/945ef7ab14dc4f9d7f40288d2df998d1837ee0888ec3659c813487572faa/pip-25.2-py3-none-any.whl.metadata
  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 19.5 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# Import PyTorch and check available devices.
# On Apple Silicon, MPS (Metal Performance Shaders) accelerates training on the GPU.
# CUDA is for NVIDIA GPUs (not available on macOS/Apple Silicon).
import torch
print("PyTorch version:", torch.__version__)
print("MPS Available:", torch.backends.mps.is_available())
print("CUDA Available:", torch.cuda.is_available())


PyTorch version: 2.8.0
MPS Available: True
CUDA Available: False


In [6]:
# Import YOLO from the ultralytics package.
# This provides the high-level `YOLO` class for training, validation, and inference.
from ultralytics import YOLO


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# Train YOLOv8 using the Apple MPS device for acceleration (if available).
# You can switch to 'yolov8s.pt' for faster training or 'yolov8n.pt' for the smallest model.
# The dataset configuration is expected at 'data.yaml' in the current directory.
# If you run this notebook from elsewhere, adjust the `data` path accordingly.
model = YOLO('yolov8m.pt')
model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device='mps'  # use 'cpu' if MPS isn't available
)


### View Training Results

After training, Ultralytics saves outputs under `runs/detect/train/`. The `results.png` plot shows losses and metrics per epoch.


In [ ]:
# Display the training curve image. If the path differs, adjust accordingly.
from IPython.display import Image
Image(filename='runs/detect/train/results.png')


In [ ]:
# Print basic training metrics available on the model object.
# Depending on the Ultralytics version, `model.metrics` may be populated after training.
try:
    metrics = model.metrics
    print(metrics)
except Exception as e:
    print("Metrics not available yet:", e)
